## This notebook does the following actions: 



 <li>Performs a segmentation</li>
 <li>Creates polygons of segmentations</li>
 <li>Extracts pixel values to segments (colours and DSM)</li>

In [8]:
import rasterio as rio
import os
import glob
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterio.features import shapes
import rasterstats

In [9]:
import skimage
from skimage import data, io 
from osgeo import gdal, ogr, osr
from skimage import exposure
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
import time
import osgeo

In [47]:
from rasterstats import zonal_stats

## Perform segmentation manually one by one

In [40]:
# define functions for the segmentation process:


def create_numpy_stack(input_stack):
    """
    creates a numpy stack from input stack
    input_stack: path to the tif with bands stack for segmentation. They must have same resolution
    type input_stack: str
    
    returns the numpy array stack
    """
    
    
    
    naip_ds = gdal.Open(input_stack)
    
    nbands = naip_ds.RasterCount
    
    
    band_data = [] # list to stack
    
    print('bands', naip_ds.RasterCount, 'rows', naip_ds.RasterYSize, 'columns',
      naip_ds.RasterXSize)
    
    for i in range(1, nbands+1):
        band = naip_ds.GetRasterBand(i).ReadAsArray()
        band_data.append(band)
    
    band_data = np.dstack(band_data)


    return band_data, naip_ds
    
    

def segmentation(input_stack,segment_params,output_ras, output_vect): 
    """
    performs a segmentation from input stack image and saves the segmentation in a raster format and a vector format
    
    input_stack: path to the tif with bands stack for segmentation. They must have same resolution
    type input_stack: str
    
    input segment_params: name and the mandatory parameters for felzenszwalb segmentation.
    doc:  https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.felzenszwalb
    type segment_params: str
    
    output_ras: where to save the raster
    type output_ras: str
    
    output_vect: where to save the vector
    type output_vect: str
    
    rtype: None, void.
    
    """
    area_st = input_stack.split("_STK")[0][-6:]
    nbands = 5
    img, naip_ds = create_numpy_stack(input_stack)
    
    # get parameters for segmentation
    scale_input_fz = int(segment_params.split("_")[1])
    sigma_input_fz = float(segment_params.split("_")[2])
    min_size_input_fz = int(segment_params.split("_")[3])
    
    ################
    #segmentation starts here
    ################
    
    driverTiff = gdal.GetDriverByName('GTiff')
    
    segments_fz = felzenszwalb(img, scale=int(scale_input_fz), sigma=1, min_size=int(min_size_input_fz))

     # save segments to raster
    segments_fn_fz = os.path.join(output_ras,"fz_"+str(scale_input_fz)+"_"+str(sigma_input_fz)+"_"+str(min_size_input_fz)+"_"+area_st+"_"+str(nbands)+".tif")
    print("Output: ", segments_fn_fz)
    segments_ds_fz = driverTiff.Create(segments_fn_fz, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                        1, gdal.GDT_Float32)
    segments_ds_fz.SetGeoTransform(naip_ds.GetGeoTransform())
    segments_ds_fz.SetProjection(naip_ds.GetProjectionRef())
    segments_ds_fz.GetRasterBand(1).WriteArray(segments_fz)
    segments_ds_fz = None
    

    mask = None



    with rio.Env():
        # GET THE REPROJECTED RASTER
        with rio.open(segments_fn_fz) as src_segments:
            image = src_segments.read(1) # first band
            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=mask, transform=src_segments.transform)))

            
    # Polygonize the segments:
    
    geoms = list(results)   
    
    gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)     
    
    gpd_polygonized_raster.crs = 'EPSG:3301'


    print("CRS of segments of polygonized segments: ",gpd_polygonized_raster.crs)
    
    #save segmentation in polygon shapes
    gpd_polygonized_raster.to_file(os.path.join(output_vect,segment_params+area_st+"_"+str(nbands))+".shp", driver="ESRI Shapefile")
    

In [41]:
# Parameters of segmentation:
segmentation_params = "fz_100_1_500_"
# Perform segmentation
segmentation(".\\raster\\54761_clip_area_3_STK.tif",
             segmentation_params,
             ".\\segmentations",  ".\\segmentations")


bands 4 rows 3937 columns 4262


I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\skimage\_shared\utils.py:394: RuntimeWarning: Got image with third dimension of 4. This image will be interpreted as a multichannel 2d image, which may not be intended.
  return func(*args, **kwargs)


Output:  .\segmentations\fz_100_1.0_500_area_3_5.tif
CRS of segments of polygonized segments:  EPSG:3301


## Pixel extraction

In [42]:
path_stack_4b = r".\raster"
path_dsm = r".\raster\dsm"
path_segments = r".\segmentations"

select_stack = [ stk for stk in glob.glob(os.path.join(path_stack_4b,"*STK.tif"))]

select_dsm = [dsm for dsm in glob.glob(os.path.join(path_dsm,"*tif"))]

select_segments_f100_500 = [ seg for seg in glob.glob(os.path.join( path_segments, "*.shp"))if "100_1_500" in seg ] # select th

In [44]:
select_segments_f100_500

['.\\segmentations\\fz_100_1_500_area_3_5.shp']

In [45]:
areas = ["area_1","area_2","area_3"]

In [48]:

for segmentation in select_segments_f100_500:
    
    gdf = gpd.read_file(segmentation)

    gdf.insert(0, 'ID_n', range(0, len(gdf)))
    gdf.drop(columns = ["raster_val"],inplace = True)

    for area in areas:
    
        if area in segmentation:
            
            with rio.open(  [ src for src in select_dsm if area in src ][0]  ) as src: # a little bit of hard coding here ...
                
                print(src)

                transf = src.transform
                data_array = src.read(1)
                src.close()
                
            
            stats = zonal_stats(gdf["geometry"], data_array, stats  = ["mean","median","percentile_95"], affine = transf)
            
            zs_df = pd.DataFrame(stats)
                
            zs_df.rename(columns={'mean':"meaDSM",
                              "median":"medDSM",
                              "percentile_95":"p95DSM"}, inplace=True)

                
            gdf = pd.concat([gdf,zs_df],axis = 1)
            
            display(gdf)
            
            gdf.to_file(os.path.join(r".\segmentations","dsm_segmentation_"+area+".shp"),driver = "ESRI Shapefile")
            
            print("--SHAPEFILE, SAVED--")
                
print("finished")

<open DatasetReader name='.\raster\dsm\area_3_dsm.tif' mode='r'>


I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


,ID_n,geometry,meaDSM,medDSM,p95DSM
0,0,"POLYGON ((662148.400 6474038.600, 662149.000 6...",52.980000,52.980000,52.980000
1,1,"POLYGON ((662303.800 6474038.600, 662304.000 6...",NaN,NaN,NaN
2,2,"POLYGON ((662079.000 6474038.400, 662079.200 6...",NaN,NaN,NaN
3,3,"POLYGON ((662323.200 6474038.400, 662323.400 6...",NaN,NaN,NaN
4,4,"POLYGON ((662339.200 6474038.400, 662339.400 6...",NaN,NaN,NaN
...,...,...,...,...,...
24352,24352,"POLYGON ((662800.200 6473329.000, 662800.600 6...",47.366247,47.430000,48.349998
24353,24353,"POLYGON ((662821.200 6473270.600, 662821.400 6...",47.557621,47.450001,48.400999
24354,24354,"POLYGON ((662839.000 6473328.400, 662839.600 6...",48.355795,48.399998,48.959999
24355,24355,"POLYGON ((662857.000 6473357.200, 662858.400 6...",48.982817,48.849998,50.309998


--SHAPEFILE, SAVED--
finished


In [52]:
select_segments_zs = [ seg for seg in glob.glob(os.path.join( path_segments, "dsm_s*.shp"))]

In [54]:

col_band = ["b","g","r","nr"]


for segmentation in select_segments_zs:
    
    gdf = gpd.read_file(segmentation)
    
    for area in areas:
        
        
        if area in segmentation:
            
            print(area)
        
            with rio.open(  [ src for src in select_stack if area in src ][0] ) as src: # a little bit of hard coding here ...
                
                print(src)
                
                n_bands = src.count
                
                transf = src.transform

                
                for band in range(1,n_bands+1): 
                    
                    data_array = src.read(band)      
            
            
                    stats = zonal_stats(gdf["geometry"], data_array, stats  = ["mean","median","percentile_95"], affine = transf)
            
                    zs_df = pd.DataFrame(stats)
                
                    zs_df.rename(columns={'mean':"mea_"+col_band[band-1],
                              "median":"med"+col_band[band-1],
                              "percentile_95":"p95"+col_band[band-1]}, inplace=True)

                
                    gdf = pd.concat([gdf,zs_df],axis = 1)
            
            src.close()
            
            display(gdf)
            
            gdf.to_file(os.path.join(r".\segmentations","zs_segmentation_"+area+"_5vars.shp"),driver = "ESRI Shapefile")
            
            print("--SHAPEFILE, SAVED--")
            
            print("...")
            
            
print("finished")

area_3
<open DatasetReader name='.\raster\54761_clip_area_3_STK.tif' mode='r'>


I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
I:\_PhD_\_y_2021_n_2022\Studies\_Data_\Process_Data\gaussian_filters\gausconda\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata expli

,ID_n,meaDSM,medDSM,p95DSM,geometry,mea_b,medb,p95b,mea_g,medg,p95g,mea_r,medr,p95r,mea_nr,mednr,p95nr
0,0,52.980000,52.980000,52.980000,"POLYGON ((662148.400 6474038.600, 662149.000 6...",130.333333,130.0,136.3,142.333333,144.0,144.0,134.333333,133.0,138.4,121.333333,122.0,122.0
1,1,NaN,NaN,NaN,"POLYGON ((662303.800 6474038.600, 662304.000 6...",114.000000,114.0,114.0,153.000000,153.0,153.0,161.000000,161.0,161.0,184.000000,184.0,184.0
2,2,NaN,NaN,NaN,"POLYGON ((662079.000 6474038.400, 662079.200 6...",90.000000,90.0,90.0,107.000000,107.0,107.0,116.000000,116.0,116.0,141.000000,141.0,141.0
3,3,NaN,NaN,NaN,"POLYGON ((662323.200 6474038.400, 662323.400 6...",89.000000,89.0,89.0,95.000000,95.0,95.0,102.000000,102.0,102.0,122.000000,122.0,122.0
4,4,NaN,NaN,NaN,"POLYGON ((662339.200 6474038.400, 662339.400 6...",92.000000,92.0,92.0,113.000000,113.0,113.0,121.000000,121.0,121.0,130.000000,130.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24352,24352,47.366247,47.430000,48.349998,"POLYGON ((662800.200 6473329.000, 662800.600 6...",139.052748,141.0,162.0,147.955537,150.0,176.0,140.397479,142.0,170.0,116.942234,118.0,150.0
24353,24353,47.557621,47.450001,48.400999,"POLYGON ((662821.200 6473270.600, 662821.400 6...",162.131660,166.0,180.0,191.315481,196.0,207.0,197.072245,201.0,212.0,188.161227,191.0,199.0
24354,24354,48.355795,48.399998,48.959999,"POLYGON ((662839.000 6473328.400, 662839.600 6...",117.574358,117.0,145.0,146.950119,147.0,177.0,152.386788,153.0,181.0,162.110378,163.0,182.0
24355,24355,48.982817,48.849998,50.309998,"POLYGON ((662857.000 6473357.200, 662858.400 6...",118.164177,119.0,132.0,149.694259,151.0,164.0,155.793820,157.0,170.0,172.111519,173.0,184.0


--SHAPEFILE, SAVED--
...
finished
